BigQuery Studio Notebook
#**GA4 Event Parameters Data Check (v2)**

**author:** Julius Selnekovic | [selnekovic.com](https://selnekovic.com)
<br><br>

###Set The Variables:

* `PROJECT_ID` – bigquery billing project id
* `DATASET_ID` – ga4 dataset id (e.g., `analytics_123456789`)
* `START_DATE` / `END_DATE` – date range in `yyyymmdd` format
* `EVENTS` – tuple of event names to include
* `TOP_VALUES_FOR_KEY` – how many top parameter values to keep per key (k)


In [ ]:
PROJECT_ID = "mycompany-marketing-dwh"
DATASET_ID = "analytics_123456789"
START_DATE = "20250801"
END_DATE = "20250901"
EVENTS = "('first_visit', 'session_start', 'page_view', 'form_submit', 'purchase')"
TOP_VALUES_FOR_KEY = 5

#How To Run

Click **Run all** from the top menu. The script will generate an **html report** and save it to your working directory.
<br><br>

###Warning
Broad date ranges and many selected events can lead to very large queries. This may:

* use significant bigquery resources,
* increase processing time,
* exhaust your notebook’s ram or cpu.

###Tip

* start with a short date range and a few events to validate the flow and expand once everything works.

In [ ]:
# Import libraries
import polars as pl
from google.cloud import bigquery
from google.colab import files
from collections import defaultdict
from typing import Optional, Dict, Any
from IPython.display import HTML, display


In [ ]:
# Query data
client = bigquery.Client()
query = f"""
SELECT
  event_name, event_params
FROM
  `{PROJECT_ID}.{DATASET_ID}.events_*`
WHERE _TABLE_SUFFIX BETWEEN '{START_DATE}' AND '{END_DATE}'
AND event_name in {EVENTS}
"""

arrow_table = client.query(query).to_arrow()
df_polars = pl.from_arrow(arrow_table)

In [ ]:
# Define extracting function
def extract_data_from_polars_vectorized(
    df: pl.DataFrame,
    top_values_per_key: int = 5,
) -> Dict[str, Dict[str, Any]]:
    """
    build {event_name: {"count": int, "params": {key: {value: freq}}}}

    args
    ----
    df : pl.DataFrame
        expects columns:
          - event_name : Utf8
          - event_params : List[Struct({"key": Utf8, "value": Struct({
                "string_value": Utf8?,"int_value": Int64?,"float_value": Float32?,
                "double_value": Float64?
            })})]
    top_values_per_key : int
        keep this many most frequent values per (event_name, key)
    """

    # total events per event_name
    event_counts = (
        df.group_by("event_name")
          .len()
          .rename({"len": "count"})
          .sort("count", descending=True)
    )

    # explode GA4 event_params
    params_long = (
        df.select("event_name", "event_params")
          .explode("event_params")
          .with_columns(param=pl.col("event_params"))
          .drop("event_params")
          .with_columns(
              key=pl.col("param").struct.field("key"),
              string_value=pl.col("param").struct.field("value").struct.field("string_value"),
              int_value=pl.col("param").struct.field("value").struct.field("int_value"),
              float_value=pl.col("param").struct.field("value").struct.field("float_value"),
              double_value=pl.col("param").struct.field("value").struct.field("double_value"),
          )
          .with_columns(
              value=pl.coalesce([
                  pl.col("string_value"),
                  pl.col("int_value").cast(pl.Utf8),
                  pl.col("float_value").cast(pl.Utf8),
                  pl.col("double_value").cast(pl.Utf8),
              ]).fill_null("null")
          )
          .select("event_name", "key", "value")
    )

    # frequencies
    value_counts = (
        params_long
        .group_by(["event_name", "key", "value"])
        .len()
        .rename({"len": "freq"})
    )

    # top-K values per (event_name, key)
    topk = (
        value_counts
        .sort(["event_name", "key", "freq"], descending=[False, False, True])
        .group_by(["event_name", "key"])
        .head(top_values_per_key)
    )

    # assemble nested dict
    counts_map = {r["event_name"]: r["count"] for r in event_counts.to_dicts()}

    extracted = defaultdict(lambda: {"count": 0, "params": defaultdict(dict)})
    for row in topk.to_dicts():
        en, k, v, c = row["event_name"], row["key"], row["value"], row["freq"]
        extracted[en]["params"][k][v] = c

    out = {}
    for en, cnt in counts_map.items():
        params = extracted[en]["params"]
        out[en] = {
            "count": int(cnt),
            "params": {k: dict(vdict) for k, vdict in params.items()},
        }

    return out



In [ ]:
# define visualization functions
def save_result_to_html(
    result: dict,
    path: str = "event_params_report.html",
    top_values_per_key: int = 5,
    open_sections: bool = False,
    page_title: str | None = None,
    project_id: str = "mycompany-marketing-dwh",
    dataset_id: str = "analytics_123456789",
    start_date: str = "20250801",
    end_date: str = "20250901",
) -> str:
    """
    renders a modern, readable HTML report:
      - centered hero title
      - metadata line (project • dataset • date range) BETWEEN title and counts
      - summary table with anchors
      - per-event cards, each param in a collapsible table
    """
    import html, re, datetime

    def esc(x) -> str:
        return html.escape(str(x))

    def slugify(s: str) -> str:
        s = re.sub(r"\s+", "-", str(s).strip().lower())
        s = re.sub(r"[^a-z0-9\-]+", "", s)
        return s or "event"

    def fmt_ymd(s: str) -> str:
        s = str(s)
        if re.fullmatch(r"\d{8}", s):
            return f"{s[0:4]}-{s[4:6]}-{s[6:8]}"
        if re.fullmatch(r"\d{4}-\d{2}-\d{2}", s):
            return s
        try:
            dt = datetime.datetime.strptime(s, "%Y%m%d")
            return dt.strftime("%Y-%m-%d")
        except Exception:
            return s

    # sort events by total count desc
    events = sorted(result.items(), key=lambda kv: kv[1].get("count", 0), reverse=True)

    # stable unique ids for anchors
    used = {}
    def unique_id(base: str) -> str:
        n = used.get(base, 0) + 1
        used[base] = n
        return base if n == 1 else f"{base}-{n}"

    # title and meta
    title_text = page_title or f"GA4 event parameters — top {top_values_per_key} values per key"
    daterange_text = f"{fmt_ymd(start_date)} - {fmt_ymd(end_date)}"

    # ------------------- HTML HEAD & START -------------------
    head = f"""<!doctype html>
<html lang="en">
<head>
<meta charset="utf-8">
<meta name="viewport" content="width=device-width,initial-scale=1">
<title>{esc(title_text)}</title>
<style>
  :root {{
    --bg: #ffffff;
    --fg: #111111;
    --muted: #666;
    --card: #ffffff;
    --border: #e6e6e6;
    --accent: #2563eb; /* blue-600 */
    --accent-soft: #eff6ff; /* blue-50 */
    --stripe: #fafafa;
    --shadow: 0 1px 2px rgba(0,0,0,.06);
    --radius: 12px;
    --sticky-h: 56px;
  }}

  * {{ box-sizing: border-box; }}
  html, body {{ margin: 0; padding: 0; background: var(--bg); color: var(--fg);
                font: 14px/1.5 system-ui, -apple-system, Segoe UI, Roboto, Arial, sans-serif; }}

  /* top bar */
  .header {{ position: sticky; top: 0; background: rgba(255,255,255,.9);
             backdrop-filter: saturate(180%) blur(8px); border-bottom: 1px solid var(--border);
             height: var(--sticky-h); z-index: 10; }}
  .header-inner {{ max-width: 1100px; height: 100%; margin: 0 auto; padding: 0 16px;
                   display: flex; align-items: center; gap: 12px; }}
  .search {{ margin-left: auto; }}
  .search input {{
    padding: 8px 12px; border: 1px solid var(--border); border-radius: 10px;
    background: var(--bg); color: var(--fg);
  }}

  .container {{ max-width: 1100px; margin: 0 auto; padding: 20px 16px 60px; }}

  /* hero */
  .hero {{ text-align: center; padding: 18px 0 8px; }}
  .hero h1 {{ font-size: 22px; margin: 0 0 6px; }}
  .sub {{ color: var(--muted); font-size: 13px; }}
  .small {{ font-size: 12px; }}

  /* cards & tables */
  .card {{ background: var(--card); border: 1px solid var(--border); border-radius: var(--radius);
           padding: 16px; margin: 14px 0; box-shadow: var(--shadow); }}
  .summary-table {{ width: 100%; border-collapse: collapse; margin-top: 6px; }}
  .summary-table th, .summary-table td {{ border-bottom: 1px solid var(--border); padding: 8px 10px; text-align: left; }}
  .summary-table tr:hover td {{ background: var(--stripe); }}
  .badge {{ display: inline-block; padding: 2px 8px; border-radius: 999px; background: var(--accent-soft); color: var(--accent);
            font-weight: 600; font-size: 12px; }}
  .muted {{ color: var(--muted); }}
  .event-card h2 {{ font-size: 18px; margin: 0; }}
  .event-head {{ display: flex; align-items: center; justify-content: space-between; gap: 8px; flex-wrap: wrap; }}

  details {{ border: 1px solid var(--border); border-radius: 10px; padding: 8px 10px; background: var(--bg); margin: 10px 0; }}
  summary {{ cursor: pointer; font-weight: 600; outline: none; }}
  .param-meta {{ margin-left: 8px; font-weight: 500; color: var(--muted); }}

  table.params {{ width: 100%; border-collapse: collapse; margin-top: 8px; }}
  table.params th, table.params td {{ border: 1px solid var(--border); padding: 8px 10px; text-align: left; }}
  table.params th {{ background: var(--stripe); }}

  /* anchor offset so sticky header doesn't cover targets */
  .anchor-target {{ scroll-margin-top: calc(var(--sticky-h) + 10px); }}

  a {{ color: var(--accent); text-decoration: none; }}
  a:hover {{ text-decoration: underline; }}
</style>
</head>
<body>
<div class="header">
  <div class="header-inner">
    <div class="muted" aria-hidden="true">report</div>
    <div class="search"><input id="filter" type="search" placeholder="filter events…" aria-label="filter events"></div>
  </div>
</div>

<div class="container">
  <div class="hero">
    <h1>{esc(title_text)}</h1>
    <!-- metadata line BETWEEN hero title and counts -->
    <div class="sub">
      project: <b>{esc(project_id)}</b> • dataset: <b>{esc(dataset_id)}</b> • date range: <b>{esc(daterange_text)}</b>
    </div>
    <div class="sub">
      {len(events)} events • generated {datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")}
    </div>
  </div>
"""
    # ------------------- SUMMARY -------------------
    summary_rows = []
    for event_name, data in events:
        eid = unique_id(f"event-{slugify(event_name)}")
        data["_html_id"] = eid
        summary_rows.append(
            f"<tr data-en='{html.escape(str(event_name)).lower()}'>"
            f"<td><a href='#{eid}'>{esc(event_name)}</a></td>"
            f"<td class='small muted'>{int(data.get('count', 0))}</td>"
            "</tr>"
        )

    summary_html = f"""
<div id="summary" class="card anchor-target">
  <h2 style="margin:0 0 6px">summary</h2>
  <p class="muted" style="margin:0 0 8px">click any event to jump to its section.</p>
  <table class="summary-table">
    <thead><tr><th>event</th><th>count</th></tr></thead>
    <tbody>
      {''.join(summary_rows)}
    </tbody>
  </table>
</div>
"""

    # ------------------- EVENT SECTIONS -------------------
    sections = []
    details_state = " open" if open_sections else ""
    for event_name, data in events:
        eid = data["_html_id"]
        count = int(data.get("count", 0))
        sections.append(f"""
<div id="{eid}" class="card event-card anchor-target">
  <div class="event-head">
    <h2>{esc(event_name)} <span class="badge">#{count} events</span></h2>
    <div><a href="#summary" title="back to summary">↑ back to summary</a></div>
  </div>
""")
        params = (data.get("params") or {})
        for key in sorted(params.keys(), key=lambda s: str(s).lower()):
            kvs = params[key] or {}
            kv_items = sorted(kvs.items(), key=lambda t: (-int(t[1]), str(t[0])))
            rows = "\n".join(
                f"<tr><td>{esc(v)}</td><td class='muted'>{int(c)}</td></tr>"
                for v, c in kv_items
            )
            sections.append(f"""
  <details{details_state}>
    <summary>{esc(key)} <span class="param-meta">({len(kv_items)} values)</span></summary>
    <table class="params">
      <thead><tr><th>value</th><th>count</th></tr></thead>
      <tbody>
        {rows}
      </tbody>
    </table>
  </details>
""")
        sections.append("</div>")  # end card

    # ------------------- FOOTER & CLOSE -------------------
    footer = """
</div>
<script>
  const filter = document.getElementById('filter');
  const rows = Array.from(document.querySelectorAll('.summary-table tbody tr'));
  filter?.addEventListener('input', (e) => {
    const q = e.target.value.toLowerCase().trim();
    rows.forEach(tr => {
      const name = tr.getAttribute('data-en') || '';
      tr.style.display = q && !name.includes(q) ? 'none' : '';
    });
  });
</script>
</body>
</html>
"""

    html_doc = head + summary_html + "\n".join(sections) + footer
    with open(path, "w", encoding="utf-8") as f:
        f.write(html_doc)
    return path

def render_result_inline(result: dict):
    parts = ["<style>h2{margin:16px 0 4px} h4{margin:8px 0} ul{margin:4px 0 12px}</style>"]
    for en, data in result.items():
        parts.append(f"<h2>EVENT NAME: {en} (#{data['count']})</h2>")
        for k, vals in data["params"].items():
            parts.append(f"<h4>KEY: {k}</h4><ul>")
            for v, c in vals.items():
                parts.append(f"<li><b>{v}</b>: {c}</li>")
            parts.append("</ul>")
        parts.append("<hr>")
    display(HTML("".join(parts)))

In [ ]:
# Do the work
result = extract_data_from_polars_vectorized(df_polars, top_values_per_key=TOP_VALUES_FOR_KEY)
# render_result_inline(result) // uncomment if you want to see inline results in notebook
path = save_result_to_html(
    result=result,
    path="event_params_report.html",
    top_values_per_key=TOP_VALUES_FOR_KEY,
    open_sections=False,
    page_title=None,  
    project_id=PROJECT_ID,
    dataset_id=DATASET_ID,
    start_date=START_DATE,
    end_date=END_DATE,
)
files.download("event_params_report.html")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>